# Crawler

This notebook contains started code structure for creating a crawler on single machine

**Author:** Noshaba Nasir

**Date:** 26/3/2021

**Updated by:** Muhammad Atta Mohio Din Noon 17L-4163
    

In [1]:
import os 
import random
import threading
import requests
import warnings
import threading
import time
import urllib.robotparser
import requests
 
from urllib.request import urlopen
from urllib.request import Request
from urllib.parse import urlsplit
from urllib.parse import urljoin
from bs4 import BeautifulSoup
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

# Add any library to be imported here

In [2]:
# Crawler Parameters
BACKQUEUES= 3
THREADS = BACKQUEUES*3
FRONTQUEUES = 5
WAITTIME = 15.0 # wait 15 seconds before fetching URLS from 
CURRENTTIME = 0.0
NEXTTIME = 0.0
TOTALPAGESCRAWLED = 0

# Add any other global parameters here

# FRONTIER

Frontier should use the Mercator frontier design as discussed in lecture.

Preferably it should be a class and should have the given functions.

*prioritizer* function is a stub right now, it will return a random number  between 1 to f for given URL 

In [3]:
class frontier:
# add the code for frontier here
# should have functions __init__, get_URL, add_URLs, add_to_backqueue
    def __init__(self, seedurl):
        '''
        Initializer
        '''
        
        self.pagescrawled = 0
        self.fq=[None] * FRONTQUEUES # list of empty frontqueues, multiplying with number of frontqueues needed
        self.bq=[None] * BACKQUEUES # list of empty backqueues, multiplying with number of backqueues needed
        self.host_list = [] # all visited websites will have their main page url saved in this list the first time they are visited
        self.robot_parsed = [] # list of all parsed robot.txt files
        self.heap = [] # is heap used for frontier
        self.lastheapnum = 2
        self.heap.append(2)
        self.heap.append(1)
        self.heap.append(0) # for intializing heap
        # for helping allocate number of queues dynamically
        for i in range (0,len(seedurl)):
            [host, self.host_list, self.robot_parsed] = check_firsttime(seedurl[i], self.host_list, self.robot_parsed)          
            priority = prioritizer(seedurl[i], FRONTQUEUES)
            q = seedurl[i]
            if self.fq[priority]==None:
                temp_q = []
                temp_q.append(q)
                self.fq[priority]=temp_q
            else:
                temp_q=[]
                temp_q=self.fq[priority]
                temp_q.append(q)
                self.fq[priority]=temp_q
        for i in range(0,BACKQUEUES):
            self.add_to_backqueue(i)

    # add more functions here
    def get_url(self):
        try:
            bqnum = self.heap.pop(0)
            self.lastheapnum = bqnum
        except:#adding more heapvalues to keep code running
            if self.lastheapnum == 0:
                self.heap.append(1)
                self.heap.append(0)
                self.heap.append(2)
                bqnum = 2
                self.lastheapnum == 2
            elif self.lastheapnum == 1:
                self.heap.append(2)
                self.heap.append(1)
                self.heap.append(0)
                bqnum = 0
                self.lastheapnum == 0
            else:
                self.heap.append(0)
                self.heap.append(2)
                self.heap.append(1)
                bqnum = 1
                self.lastheapnum == 1
            try:
                crawl = self.bq[bqnum].pop(0)
                self.add_to_backqueue(bqnum)
                self.heap.append(bqnum)
                NEXTTIME = CURRENTTIME + WAITTIME
                return crawl
            except:
                if self.bq[bqnum] == None or not self.bq[bqnum]:
                    self.add_to_backqueue(bqnum)
                    NEXTTIME = CURRENTTIME + WAITTIME
                    return self.get_url()
            
    # adding urls to front queue
    def add_urls(self,urls):
        global TOTALPAESCRAWLED
        if TOTALPAGESCRAWLED < 100:
            for i in urls:
                [host, self.host_list, self.robot_parsed] = check_firsttime(i, self.host_list, self.robot_parsed)
                val = self.host_list.index(host)                    
                robo = self.robot_parsed[val]
                if check_robots(i,robo):
                    priority = prioritizer(i,FRONTQUEUES)
                    if self.fq[priority]==None or len(self.fq[priority])==0:
                        temp_q = []
                        temp_q.append(i)
                        self.fq[priority]=temp_q
                        for i in range(0,BACKQUEUES):
                            self.add_to_backqueue(i)
                    else:
                        temp_q=[]
                        temp_q=self.fq[priority]
                        temp_q.append(i)
                        self.fq[priority]=temp_q
                        for i in range(0,BACKQUEUES):
                            self.add_to_backqueue(i)
            
        
     
    # adding urls to back queue
    def add_to_backqueue(self,bqnum):
        # 0 has lowest priority, 4 has higest priority, thus largest number eg: [0,1,1,2,2,2,3,3,3,3,4,4,4,4,4]
        arr = []
        for i in range(0,FRONTQUEUES):
            for j in range(0,i+1):
                arr.append(i)
        r = random.randint(0,len(arr)-1)
        r = arr[r]
        exists = False
        while self.bq[bqnum] == None or not self.bq[bqnum]:
            while self.fq[r] == None or not self.fq[r]:
                r = random.randint(0,len(arr)-1)
                r = arr[r]
            url = self.fq[r].pop(0)
            host_url = "{0.scheme}://{0.netloc}".format(urlsplit(url))
            for i in self.bq:
                if i != None:
                    if url in i:
                        self.bq[i].append(url)
                        
                        exists = true
            if not exists:
                if self.bq[bqnum] != None:
                    temp_q=[]
                    temp_q=self.bq[bqnum]
                    temp_q.append(url)
                    self.bq[bqnum]=temp_q
                else:
                    temp_q = []
                    temp_q.append(url)
                    self.bq[bqnum]=temp_q 
    def print(self):
        for j in self.bq:
            print(j)
        print('---------------------')
        
            

# priority assignment function that returns random value
def prioritizer(URL,f):
    """
    Take URL and returns priority from 1 to F
    Right now it like a stub function.
    It will return a random number from 1 to f for given inputs.
    """
    return random.randint(0,f-1) 


# FILTER URLS

Filter the URLS that are in robots.txt files of server and the have been already processed. 

In [4]:
# write code for filtering urls here 

# checks if page is allowed to be crawled or not
def check_robots(urll,robot_parse):
    return robot_parse.can_fetch('*',urll)


# removes duplicate urls
def remove_duplicates(urlarr):
    arr=[]
    for i in range(0,len(urlarr)):
        urll = urlarr.pop(0)
        if not urll in arr:
            arr.append(urll)
    return arr


def crawl_page(urll, tid):
    # crawls page, gets all data, calls functions to remove duplicates and parses any site that is not properlly written
    print('crawling ' + str(tid))
    
    global TOTALPAGESCRAWLED
    try:
        response = urlopen(urll)
        html = response.read()
        soup = BeautifulSoup(html, 'lxml')
        html2 = requests.get(urll)
        soup2 = BeautifulSoup(html2.content, 'lxml')
    except:
        return None
    
    #data = str(soup.findAll(text=True)).replace('\\n','')
    filename = str(TOTALPAGESCRAWLED) + '.txt' #name of txt file would be page number crawled
    #output_file = open(filename, 'a')
    #output_file.write(data )
    with open(filename, 'w', encoding='utf-8') as f_out:
        f_out.write(soup.prettify())
    print(filename + "is being written")
    f_out.close()
    TOTALPAGESCRAWLED = TOTALPAGESCRAWLED + 1
    
    arr = []
    host_url = "{0.scheme}://{0.netloc}".format(urlsplit(urll))
    for link in soup.find_all('a'):
        link = urljoin(host_url,link.get('href')) # to get the proper parsed url, not something withput domain name
        arr.append(link)
    arr=remove_duplicates(arr)
    return arr
    
#checks if url is visited first time ever or not
def check_firsttime(urll, hlist, robot_parsed):
    # Checks if host first time visited by the crawler
    # if yes, then go to robots.txt and list all urls that are not to be crawled upon
    host_url = "{0.scheme}://{0.netloc}".format(urlsplit(urll))
    if host_url in hlist:
        return [host_url, hlist, robot_parsed]
    else:
        
        hurl = host_url + '/'
        parser = urllib.robotparser.RobotFileParser()
        parser.set_url(urljoin(hurl, 'robots.txt'))
        try:
            parser.read() 
            hlist.append(host_url)
            robot_parsed.append(parser)
        except:
            hlist.append(host_url)
            robot_parsed.append(parser)
        return [host_url, hlist, robot_parsed]


## ---------------------Write any other codes/data require to run the crawler above this cell-----------

# Run Crawler

In [5]:
# Theard task
class mythread(threading.Thread):
    def __init__(self, TID,frontier):
        threading.Thread.__init__(self)
        self.TID = TID
        self.frontier = frontier
    def run(self):
        print("starting thread number: " + str(self.TID))
        crawler_thread_task(self.frontier,self.TID)
        print("ending thread number: " + str(self.TID))

def crawler_thread_task(frontier,tid):
    # define individual crawler thread's function here as studies in class
    global TOTALPAGESCRAWLED
    while TOTALPAGESCRAWLED < 100:
        CURRENTTIME = time.time()
        global NEXTTIME
        if CURRENTTIME >= NEXTTIME:
            urll = frontier.get_url()
            #print(urll)
            while urll == None:
                frontier.print()
                for i in range(0,BACKQUEUES):
                    frontier.add_to_backqueue(i)
                urll = frontier.get_url()
                #print(urll)
            arr = crawl_page(urll,tid)
            if arr != None:
                frontier.add_urls(arr)
            

        else:
            while(CURRENTTIME < NEXTTIME):
                CURRENTTIME = time.time()
        

In [6]:
# intialize every thing 
seedurl=[]
seedurl.append('https://docs.oracle.com/en/')
seedurl.append('https://www.oracle.com/corporate/')
seedurl.append('https://en.wikipedia.org/wiki/Machine_learning')
seedurl.append('https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html')
seedurl.append('https://docs.oracle.com/middleware/jet210/jet/index.html')
seedurl.append('https://en.wikipedia.org/w/api.php')
seedurl.append('https://en.wikipedia.org/api/')
seedurl.append('https://en.wikipedia.org/wiki/Weka_(machine_learning)')

In [ ]:
# start the threads

myfrontier = frontier(seedurl)
threads = []
for i in range(0,THREADS):
    th = mythread(i,myfrontier)
    threads.append(th)
    threads[i].start()
for j in range(0,THREADS):
        threads[j].join()

starting thread number: 0starting thread number: 1
['https://en.wikipedia.org/wiki/Machine_learning']
['https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html']
['https://docs.oracle.com/en/']
---------------------
['https://en.wikipedia.org/wiki/Machine_learning']
['https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html']
['https://docs.oracle.com/en/']
---------------------
['https://en.wikipedia.org/wiki/Machine_learning']
['https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html']
['https://docs.oracle.com/en/']
---------------------
crawling 1

['https://en.wikipedia.org/wiki/Machine_learning']starting thread number: 2
['https://en.wikipedia.org/wiki/Machine_learning']
['https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html']
['https://en.wikipedia.org/w/api.php']
---------------------
['https://en.wikipedia.org/wiki/Machine_learning']
['https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html']
['https://en.wikipedia.org/w/api.php']
---------------------
['https://en.wikipedia.org/wiki/Machi

['https://docs.oracle.com#BEGIN', 'https://docs.oracle.com/en/middleware/', 'https://docs.oracle.com/jet_tasks.html', 'https://docs.oracle.com/jet_video.html', 'https://docs.oracle.com/jet_books.html', 'https://docs.oracle.com/developer/GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1.htm#JETDG-GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1']
['https://docs.oracle.com/jet_tutorials.html']
['https://docs.oracle.com/developer/GUID-ABE2373F-287F-4C3A-BEBD-02F179F399FD.htm#JETDG115']
---------------------
['https://docs.oracle.com#BEGIN', 'https://docs.oracle.com/en/middleware/', 'https://docs.oracle.com/jet_tasks.html', 'https://docs.oracle.com/jet_video.html', 'https://docs.oracle.com/jet_books.html', 'https://docs.oracle.com/developer/GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1.htm#JETDG-GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1']
['https://docs.oracle.com/jet_tutorials.html']
['https://docs.oracle.com/developer/GUID-ABE2373F-287F-4C3A-BEBD-02F179F399FD.htm#JETDG115']
---------------------
['https://

['https://docs.oracle.com/en/middleware/', 'https://docs.oracle.com/jet_tasks.html', 'https://docs.oracle.com/jet_video.html', 'https://docs.oracle.com/jet_books.html', 'https://docs.oracle.com/developer/GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1.htm#JETDG-GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1']
['https://foundation.wikimedia.org/wiki/Developer_app_guidelines', 'https://www.csie.ntu.edu.tw/libsvmtools']
['https://www.csie.ntu.edu.tw/index.html', 'https://www.csie.ntu.edu.tw/papers/guide/guide.pdf', 'https://www.csie.ntu.edu.tw/libsvmtools/datasets']
---------------------
['https://docs.oracle.com/en/middleware/', 'https://docs.oracle.com/jet_tasks.html', 'https://docs.oracle.com/jet_video.html', 'https://docs.oracle.com/jet_books.html', 'https://docs.oracle.com/developer/GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1.htm#JETDG-GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1']
['https://foundation.wikimedia.org/wiki/Developer_app_guidelines', 'https://www.csie.ntu.edu.tw/libsvmtools']
['https:/

['https://docs.oracle.com/jet_video.html', 'https://docs.oracle.com/jet_books.html', 'https://docs.oracle.com/developer/GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1.htm#JETDG-GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1']
['https://www.csie.ntu.edu.tw/libsvmtools']
['https://www.csie.ntu.edu.tw/papers/guide/guide.pdf', 'https://www.csie.ntu.edu.tw/libsvmtools/datasets']
---------------------
['https://docs.oracle.com/jet_video.html', 'https://docs.oracle.com/jet_books.html', 'https://docs.oracle.com/developer/GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1.htm#JETDG-GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1']
['https://www.csie.ntu.edu.tw/libsvmtools']
['https://www.csie.ntu.edu.tw/papers/guide/guide.pdf', 'https://www.csie.ntu.edu.tw/libsvmtools/datasets']
---------------------
['https://docs.oracle.com/jet_video.html', 'https://docs.oracle.com/jet_books.html', 'https://docs.oracle.com/developer/GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1.htm#JETDG-GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1']
['http

12.txtis being written
['https://docs.oracle.com/developer/GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1.htm#JETDG-GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1']
['https://en.wikipedia.org#searchInput']
['https://en.wikipedia.org#mw-head']
---------------------
['https://docs.oracle.com/developer/GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1.htm#JETDG-GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1']
['https://en.wikipedia.org#searchInput']
['https://en.wikipedia.org#mw-head']
---------------------
['https://docs.oracle.com/developer/GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1.htm#JETDG-GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1']
['https://en.wikipedia.org#searchInput']
['https://en.wikipedia.org#mw-head']
---------------------
['https://docs.oracle.com/developer/GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1.htm#JETDG-GUID-4D5A4D37-3CF6-41A0-BED2-EC14664645A1']
['https://en.wikipedia.org#searchInput']
['https://en.wikipedia.org#mw-head']
---------------------
crawling 7
13.txtis being written
14.txtis being 

...

['https://en.wikipedia.org/wiki/University_of_Waikato']
['https://www.oracle.com/corporate/citizenship/']
['https://www.oracle.com#industries']
---------------------
['https://en.wikipedia.org/wiki/University_of_Waikato']
['https://www.oracle.com/corporate/citizenship/']
['https://www.oracle.com#industries']
---------------------
['https://en.wikipedia.org/wiki/University_of_Waikato']
['https://www.oracle.com/corporate/citizenship/']
['https://www.oracle.com#industries']
---------------------
['https://en.wikipedia.org/wiki/University_of_Waikato']
['https://www.oracle.com/corporate/citizenship/']
['https://www.oracle.com#industries']
---------------------
crawling 2
['https://en.wikipedia.org/wiki/University_of_Waikato']
['https://en.wikipedia.org/wiki/Statistical_learning_in_language_acquisition']
['https://www.oracle.com#industries']
---------------------
['https://en.wikipedia.org/wiki/University_of_Waikato']
['https://en.wikipedia.org/wiki/Statistical_learning_in_language_acquisiti

46.txtis being written
47.txtis being written
['https://en.wikipedia.org/wiki/Random_forest']
['https://en.wikipedia.org/wiki/Ensemble_learning']
['https://en.wikipedia.org#searchInput']
---------------------
['https://en.wikipedia.org/wiki/Random_forest']
['https://en.wikipedia.org/wiki/Ensemble_learning']
['https://en.wikipedia.org#searchInput']
---------------------
['https://en.wikipedia.org/wiki/Random_forest']
['https://en.wikipedia.org/wiki/Ensemble_learning']
['https://en.wikipedia.org#searchInput']
---------------------
['https://en.wikipedia.org/wiki/Random_forest']
['https://en.wikipedia.org/wiki/Ensemble_learning']
['https://en.wikipedia.org#searchInput']
---------------------
crawling 1
48.txtis being written
['https://en.wikipedia.org/wiki/Random_forest']
['https://en.wikipedia.org/wiki/Ensemble_learning']
['https://www.oracle.com/corporate/careers/culture/who-we-are.html']
---------------------
['https://en.wikipedia.org/wiki/Random_forest']
['https://en.wikipedia.org/wi

58.txtis being written
['https://en.wikipedia.org/wiki/Structured_prediction']
['https://en.wikipedia.org/wiki/Hierarchical_clustering']
['https://en.wikipedia.org/wiki/Support-vector_machine']
---------------------
['https://en.wikipedia.org/wiki/Structured_prediction']
['https://en.wikipedia.org/wiki/Hierarchical_clustering']
['https://en.wikipedia.org/wiki/Support-vector_machine']
---------------------
['https://en.wikipedia.org/wiki/Structured_prediction']
['https://en.wikipedia.org/wiki/Hierarchical_clustering']
['https://en.wikipedia.org/wiki/Support-vector_machine']
---------------------
['https://en.wikipedia.org/wiki/Structured_prediction']
['https://en.wikipedia.org/wiki/Hierarchical_clustering']
['https://en.wikipedia.org/wiki/Support-vector_machine']
---------------------
crawling 4
59.txtis being written
['https://en.wikipedia.org/wiki/Principal_component_analysis']
['https://en.wikipedia.org/wiki/Hierarchical_clustering']
['https://en.wikipedia.org/wiki/Support-vector_mac

69.txtis being written
['https://en.wikipedia.org/wiki/U-Net']
['https://en.wikipedia.org#mw-head']
['https://en.wikipedia.org/wiki/Electrochemical_RAM']
---------------------
['https://en.wikipedia.org/wiki/U-Net']
['https://en.wikipedia.org#mw-head']
['https://en.wikipedia.org/wiki/Electrochemical_RAM']
---------------------
['https://en.wikipedia.org/wiki/U-Net']
['https://en.wikipedia.org#mw-head']
['https://en.wikipedia.org/wiki/Electrochemical_RAM']
---------------------
['https://en.wikipedia.org/wiki/U-Net']
['https://en.wikipedia.org#mw-head']
['https://en.wikipedia.org/wiki/Electrochemical_RAM']
---------------------
crawling 4
70.txtis being written
['https://en.wikipedia.org/wiki/U-Net']
['https://en.wikipedia.org/wiki/Statistical_learning_theory']
['https://en.wikipedia.org/wiki/Electrochemical_RAM']
---------------------
['https://en.wikipedia.org/wiki/U-Net']
['https://en.wikipedia.org/wiki/Statistical_learning_theory']
['https://en.wikipedia.org/wiki/Electrochemical_RAM

80.txtis being written
['https://en.wikipedia.org/wiki/Association_rule_learning']
['https://en.wikipedia.org/wiki/Canonical_correlation']
['https://en.wikipedia.org/wiki/Automated_machine_learning']
---------------------
['https://en.wikipedia.org/wiki/Association_rule_learning']
['https://en.wikipedia.org/wiki/Canonical_correlation']
['https://en.wikipedia.org/wiki/Automated_machine_learning']
---------------------
['https://en.wikipedia.org/wiki/Association_rule_learning']
['https://en.wikipedia.org/wiki/Canonical_correlation']
['https://en.wikipedia.org/wiki/Automated_machine_learning']
---------------------
['https://en.wikipedia.org/wiki/Association_rule_learning']
['https://en.wikipedia.org/wiki/Canonical_correlation']
['https://en.wikipedia.org/wiki/Automated_machine_learning']
---------------------
crawling 4
81.txtis being written
['https://en.wikipedia.org/wiki/Association_rule_learning']
['https://en.wikipedia.org/wiki/Canonical_correlation']
['https://en.wikipedia.org/wiki

## ------------------------------------------------------End of Notebook---------------------------------------------------